In [ ]:
from google.colab import drive
drive.mount('/content/google_drive')

Mounted at /content/google_drive


In [ ]:
!nvidia-smi

In [ ]:
!pip install -U "tensorflow>=2.5"

In [ ]:
import tensorflow as tf

In [ ]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen


# import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

In [ ]:
import re
from shutil import copyfile
import argparse
import math
import random

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

In [ ]:
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 79836, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 79836 (delta 5), reused 1 (delta 0), pack-reused 79824
Receiving objects: 100% (79836/79836), 594.01 MiB | 16.48 MiB/s, done.
Resolving deltas: 100% (56848/56848), done.


In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cd /content
git clone https://github.com/cocodataset/cocoapi.git
cd /content/cocoapi/PythonAPI
make
cp -r pycocotools /content/models/research/
cd /content/models/research/
cp object_detection/packages/tf2/setup.py .
python -m pip install .
cd /content

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
output_directory = '/content/tf_lite'

In [ ]:
!rm -r "/content/tf_lite"

In [ ]:
!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
--trained_checkpoint_dir="/content/google_drive/MyDrive/Tensorflow_OD_API/efficientdet_d0_tflite/workspace/my_model1/checkpoint" \
--output_directory='/content/tf_lite' \
--pipeline_config_path="/content/google_drive/MyDrive/Tensorflow_OD_API/efficientdet_d0_tflite/workspace/my_model1/pipeline.config"

In [ ]:
# !python /content/models/research/object_detection/export_tflite_graph_tf2.py \
# --trained_checkpoint_dir="/content/google_drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/exported-models/saved_efficientDet_D1/checkpoint" \
# --output_directory='/content/tf_lite2' \
# --pipeline_config_path="/content/google_drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/exported-models/saved_efficientDet_D1/pipeline.config"

In [ ]:
!ls -lah /content/tf_lite/saved_model/

total 7.9M
drwxr-xr-x 4 root root 4.0K Dec 19 03:14 .
drwxr-xr-x 3 root root 4.0K Dec 19 03:14 ..
drwxr-xr-x 2 root root 4.0K Dec 19 03:14 assets
-rw-r--r-- 1 root root   57 Dec 19 03:14 fingerprint.pb
-rw-r--r-- 1 root root 7.9M Dec 19 03:14 saved_model.pb
drwxr-xr-x 2 root root 4.0K Dec 19 03:14 variables


In [ ]:
!tflite_convert --saved_model_dir=/content/tf_lite/saved_model/ --output_file=tf_lite/model.tflite

2022-12-19 06:27:21.963618: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-12-19 06:27:21.963719: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-12-19 06:27:21.963740: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2022-12-19 06:27:25.043643: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environme

In [ ]:
!ls -lah tf_lite/model.tflite

-rw-r--r-- 1 root root 195M Dec 19 03:15 tf_lite/model.tflite


In [ ]:
# based on https://github.com/tensorflow/examples/blob/master/lite/examples/object_detection/raspberry_pi/detect_picamera.py

from PIL import Image, ImageDraw, ImageFont
import re, glob
from six import BytesIO
import numpy as np
%matplotlib inline

def draw_image(image, results, size):
    result_size = len(results)
    for idx, obj in enumerate(results):
        # Prepare image for drawing
        draw = ImageDraw.Draw(image)

        # Prepare boundary box
        xmin, ymin, xmax, ymax = obj['bounding_box']
        xmin = int(xmin * size[1])
        xmax = int(xmax * size[1])
        ymin = int(ymin * size[0])
        ymax = int(ymax * size[0])

        # Draw rectangle to desired thickness
        for x in range( 0, 4 ):
            draw.rectangle((ymin, xmin, ymax, xmax), outline=(255, 255, 0), width=5)

    displayImage = np.asarray( image )
    display(Image.fromarray(displayImage))

def load_labels(path):
    """Loads the labels file. Supports files with or without index numbers."""
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        labels = {}
        for row_number, content in enumerate(lines):
            pair = re.split(r'[:\s]+', content.strip(), maxsplit=1)
            if len(pair) == 2 and pair[0].strip().isdigit():
                labels[int(pair[0])] = pair[1].strip()
            else:
                labels[row_number] = pair[0].strip()
    return labels

def set_input_tensor(interpreter, image):
    """Sets the input tensor."""
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = image


def get_output_tensor(interpreter, index):
    """Returns the output tensor at the given index."""
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor


def detect_objects(interpreter, image, threshold):
    """Returns a list of detection results, each a dictionary of object info."""
    set_input_tensor(interpreter, image)
    interpreter.invoke()

    # Get all output details
    scores = get_output_tensor(interpreter, 0)   #0:scores
    boxes = get_output_tensor(interpreter, 1) #1:boxes
    count = int(get_output_tensor(interpreter, 2)) #2:count
    classes = get_output_tensor(interpreter, 3) #3:classes
    # what = get_output_tensor(interpreter, 4) #3:classes
    # count = int(get_output_tensor(interpreter, 3))
    print("boxes=", boxes)
    print("classes=", classes)
    print("scores=", scores)
    print("count=", count)
    # print("what=", what)

    results = []
    for i in range(count):
        if scores[i] >= threshold:
            result = {
                'bounding_box': boxes[i],
                'class_id': classes[i],
                'score': scores[i]
            }
            results.append(result)
    
    return results

In [ ]:
interpreter = tf.lite.Interpreter(model_path="tf_lite/model.tflite")
interpreter.allocate_tensors()
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

In [ ]:
!pwd

/content


In [ ]:
!ls -al /content/testimages/

total 584
drwxr-xr-x 2 root root   4096 Dec 19 03:20 .
drwxr-xr-x 1 root root   4096 Dec 19 06:26 ..
-rw-r--r-- 1 root root 494356 Dec 19 03:20 abyssinian.jpg
-rw-r--r-- 1 root root  92353 Dec 19 03:20 Bulldog.jpg


In [ ]:
input_mean = 0
input_std = 1

for i, image_path in enumerate(glob.glob('/content/testimages/*.jpg')):
  print(image_path)
  image = Image.open(image_path)
  image_pred = image.resize((input_width ,input_height), Image.ANTIALIAS)
  if interpreter.get_input_details()[0]['dtype'] == np.float32:
    image_pred = (np.float32(image_pred) - input_mean) / input_std
  results = detect_objects(interpreter, image_pred, 0.2)
  print(results)

  draw_image(image, results, image.size)

In [ ]:
from object_detection.utils import label_map_util

In [ ]:
_ODT_LABEL_MAP_PATH = '/content/google_drive/MyDrive/Tensorflow_OD_API/efficientdet_d0_tflite/workspace/my_model1/label_map.pbtxt'
_TFLITE_LABEL_PATH = "/content/label_map.txt"

category_index = label_map_util.create_category_index_from_labelmap(
    _ODT_LABEL_MAP_PATH)
f = open(_TFLITE_LABEL_PATH, 'w')
for class_id in range(1, 38):   ###############
  if class_id not in category_index:
    f.write('???\n')
    continue
  name = category_index[class_id]['name']
  f.write(name+'\n')
f.close()

In [ ]:
# !pip install tf-nightly -q

In [ ]:
# !pip install tflite-support

In [ ]:
!pip install tflite_support_nightly

In [ ]:
# !pip install -U flatbuffers

In [ ]:
# import flatbuffers

In [ ]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

ObjectDetectorWriter = object_detector.MetadataWriter

In [ ]:
!ls -al /content/tf_lite/

total 198976
drwxr-xr-x 3 root root      4096 Dec 19 03:15 .
drwxr-xr-x 1 root root      4096 Dec 19 06:26 ..
-rw-r--r-- 1 root root 203736792 Dec 19 06:27 model.tflite
drwxr-xr-x 4 root root      4096 Dec 19 06:27 saved_model


In [ ]:
_MODEL_PATH="/content/tf_lite/model.tflite"

In [ ]:
_TFLITE_MODEL_WITH_METADATA_PATH = '/content/model_with_metadata.tflite'

In [ ]:
_TFLITE_LABEL_PATH="/content/label_map.txt"

In [ ]:
writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [127.5], [127.5],[_TFLITE_LABEL_PATH])

In [ ]:
writer_utils.save_file(writer.populate(), _TFLITE_MODEL_WITH_METADATA_PATH)

In [ ]:
from tflite_support import metadata

In [ ]:
displayer = metadata.MetadataDisplayer.with_model_file(_TFLITE_MODEL_WITH_METADATA_PATH)

In [ ]:
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

In [ ]:
displayer = metadata.MetadataDisplayer.with_model_file("/content/efficientdet-lite1.tflite")

In [ ]:
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

In [ ]:
import flatbuffers
import os
from tensorflow_lite_support.metadata import metadata_schema_py_generated as _metadata_fb
from tensorflow_lite_support.metadata.python import metadata as _metadata
from tensorflow_lite_support.metadata.python.metadata_writers import metadata_info
from tensorflow_lite_support.metadata.python.metadata_writers import metadata_writer
from tensorflow_lite_support.metadata.python.metadata_writers import writer_utils

In [ ]:
model_meta = _metadata_fb.ModelMetadataT()

In [ ]:
model_meta.name="efficientdet_d1_oxford_pet_detector"

In [ ]:
model_meta.description = (
    "Identify which of a known set of objects might be present and provide "
    "information about their positions within the given image or a video "
    "stream.")

In [ ]:
# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = "image"
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

In [ ]:
# Creates outputs info.
output_location_meta = _metadata_fb.TensorMetadataT()
output_location_meta.name = "location"
output_location_meta.description = "The locations of the detected boxes."
output_location_meta.content = _metadata_fb.ContentT()
output_location_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.BoundingBoxProperties)
output_location_meta.content.contentProperties = (
    _metadata_fb.BoundingBoxPropertiesT())
output_location_meta.content.contentProperties.index = [1, 0, 3, 2]
output_location_meta.content.contentProperties.type = (
    _metadata_fb.BoundingBoxType.BOUNDARIES)
output_location_meta.content.contentProperties.coordinateType = (
    _metadata_fb.CoordinateType.RATIO)
output_location_meta.content.range = _metadata_fb.ValueRangeT()
output_location_meta.content.range.min = 2
output_location_meta.content.range.max = 2

In [ ]:
output_class_meta = _metadata_fb.TensorMetadataT()
output_class_meta.name = "category"
output_class_meta.description = "The categories of the detected boxes."
output_class_meta.content = _metadata_fb.ContentT()
output_class_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_class_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
output_class_meta.content.range = _metadata_fb.ValueRangeT()
output_class_meta.content.range.min = 2
output_class_meta.content.range.max = 2
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename("label_map.txt")
label_file.description = "Label of objects that this model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_VALUE_LABELS
output_class_meta.associatedFiles = [label_file]

In [ ]:
output_score_meta = _metadata_fb.TensorMetadataT()
output_score_meta.name = "score"
output_score_meta.description = "The scores of the detected boxes."
output_score_meta.content = _metadata_fb.ContentT()
output_score_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_score_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
output_score_meta.content.range = _metadata_fb.ValueRangeT()
output_score_meta.content.range.min = 2
output_score_meta.content.range.max = 2

In [ ]:
output_number_meta = _metadata_fb.TensorMetadataT()
output_number_meta.name = "number of detections"
output_number_meta.description = "The number of the detected boxes."
output_number_meta.content = _metadata_fb.ContentT()
output_number_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_number_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())


In [ ]:
# Creates subgraph info.
group = _metadata_fb.TensorGroupT()
group.name = "detection result"
group.tensorNames = [
    output_location_meta.name, output_class_meta.name,
    output_score_meta.name
]
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [
    output_location_meta, output_class_meta, output_score_meta,
    output_number_meta
]
subgraph.outputTensorGroups = [group]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

In [ ]:
populator = _metadata.MetadataPopulator.with_model_file("/content/model_with_metadata.tflite")
populator.load_metadata_buffer(metadata_buf)
# populator.load_associated_files(["/content/label_map.txt"])
populator.populate()

In [ ]:
displayer = metadata.MetadataDisplayer.with_model_file(_TFLITE_MODEL_WITH_METADATA_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

In [ ]:
displayer = _metadata.MetadataDisplayer.with_model_file(export_model_path)
export_json_file = os.path.join(FLAGS.export_directory,
                    os.path.splitext(model_basename)[0] + ".json")
json_file = displayer.get_metadata_json()
# Optional: write out the metadata as a json file
with open(export_json_file, "w") as f:
  f.write(json_file)

In [ ]:
interpreter = tf.lite.Interpreter(model_path="/content/model_with_metadata.tflite")
interpreter.allocate_tensors()
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

In [ ]:
input_mean = 0
input_std = 1

for i, image_path in enumerate(glob.glob('/content/testimages/*.jpg')):
  print(image_path)
  image = Image.open(image_path)
  image_pred = image.resize((input_width ,input_height), Image.ANTIALIAS)
  print(np.float32(image_pred))
  print("*" * 60)
  if interpreter.get_input_details()[0]['dtype'] == np.float32:
    image_pred = (np.float32(image_pred) - input_mean) / input_std
    print(image_pred)
  results = detect_objects(interpreter, image_pred, 0.2)
  print(results)

  draw_image(image, results, image.size)